In [96]:
# Paste output from benchmark binary. This notebook parses and plots the benchmark results.
benchmark_output = '''
-------------------------------------------------------------------------------------------------
Benchmark                                       Time             CPU   Iterations UserCounters...
-------------------------------------------------------------------------------------------------
ScanSpeed                                16435614 ns     16430875 ns           40 bytes_per_second=2.97173G/s
SplitLines/strchr                        45263994 ns     45243800 ns           15 bytes_per_second=1105.12M/s
SplitLines/find_first_of(char)           36774316 ns     36749474 ns           19 bytes_per_second=1.32868G/s
SplitLines/find_first_of(str)            38361035 ns     38343333 ns           18 bytes_per_second=1.27345G/s
SplitLines/getline                      217629236 ns    217183667 ns            3 bytes_per_second=230.22M/s
LineTokenize/strpbrk/space_only              51.8 ns         51.7 ns     13555383 bytes_per_second=774.7M/s lines_tokenized_per_second=58.0237M/s
LineTokenize/strpbrk/space_tab               87.4 ns         87.4 ns      8041171 bytes_per_second=458.325M/s lines_tokenized_per_second=34.3278M/s
LineTokenize/strtok/space_only               71.2 ns         71.2 ns      9858322 bytes_per_second=562.863M/s lines_tokenized_per_second=42.1574M/s
LineTokenize/strtok/space_tab                 108 ns          108 ns      6590530 bytes_per_second=370.697M/s lines_tokenized_per_second=27.7646M/s
IntFieldParse/from_chars                     10.3 ns         10.3 ns     65245556 bytes_per_second=647.387M/s fields_converted_per_second=193.953M/s
IntFieldParse/atol                           16.6 ns         16.5 ns     41809016 bytes_per_second=403.678M/s fields_converted_per_second=120.939M/s
IntFieldParse/stoll                          39.8 ns         39.8 ns     17590282 bytes_per_second=167.739M/s fields_converted_per_second=50.2536M/s
IntFieldParse/strtoll                        18.8 ns         18.8 ns     37385174 bytes_per_second=355.464M/s fields_converted_per_second=106.495M/s
IntFieldParse/sscanf                         69.1 ns         69.1 ns     10143164 bytes_per_second=96.6652M/s fields_converted_per_second=28.9602M/s
IntFieldParse/istringstream                   220 ns          220 ns      3187077 bytes_per_second=30.3825M/s fields_converted_per_second=9.10238M/s
DoubleFieldParse/strtod                      50.4 ns         50.4 ns     13913458 bytes_per_second=265.102M/s fields_converted_per_second=59.5672M/s
DoubleFieldParse/sscanf                       187 ns          187 ns      3742515 bytes_per_second=71.4503M/s fields_converted_per_second=16.0545M/s
DoubleFieldParse/istringstream                391 ns          391 ns      1785851 bytes_per_second=34.1111M/s fields_converted_per_second=7.66459M/s
LineParse/istringstream                       955 ns          954 ns       735449 bytes_per_second=41.985M/s lines_parsed_per_second=3.1446M/s
LineParse/sscanf                              360 ns          359 ns      1935964 bytes_per_second=111.423M/s lines_parsed_per_second=8.34543M/s
LineParse/strtoll+strtod                      111 ns          111 ns      6313758 bytes_per_second=359.969M/s lines_parsed_per_second=26.9611M/s
LineParse/from_chars+strtod/space_only        104 ns          104 ns      6760933 bytes_per_second=386.739M/s lines_parsed_per_second=28.9661M/s
LineParse/from_chars+strtod/space_tab         104 ns          104 ns      6697283 bytes_per_second=384.586M/s lines_parsed_per_second=28.8048M/s
BlockParse/istringstream               1093338125 ns   1092352000 ns            1 bytes_per_second=45.7728M/s
BlockParse/from_chars+strtod            165292396 ns    165256500 ns            4 bytes_per_second=302.56M/s
'''

In [97]:
def parse_float_with_magnitude(f: str):
    mags = {
        "K": 10**3,
        "M": 10**6,
        "G": 10**9,
        "T": 10**12,
    }
    mag = mags.get(f[-1], 1)
    if mag != 1:
        f = f[:-1]
    return float(f) * mag


def parse_google_benchmark_output(gbm):
    import jc

    # Add an extra space between column headers. If there's only one space then jc will parse it as one column.
    gbm = gbm.replace("Iterations UserCounters", "Iterations  UserCounters")
    res = jc.parse("asciitable", gbm, raw=True)

    for row in res:
        # Break out the benchmark variants
        bench = row["Benchmark"]
        i = bench.find("/")
        if i > 0:
            row["Group"] = bench[0:i]
            row["Variant"] = bench[i+1:]
        else:
            row["Group"] = bench

        # Parse the user counters
        if "UserCounters" in row:
            counters = row["UserCounters"].split(" ")
            for counter in counters:
                k, v = counter.split("=")
                v = v.removesuffix("/s")
                row[k] = parse_float_with_magnitude(v)
    return res


In [98]:
import pandas as pd
import plotly.express as px

df = pd.DataFrame(parse_google_benchmark_output(benchmark_output))
df.sort_values(by='bytes_per_second', ascending=False, inplace=True)

In [100]:
fig = px.bar(df[df.Group=="IntFieldParse"], x='Variant', y='bytes_per_second', title="String to Integer")
fig.update_layout(
    yaxis = dict(
        title=None,
        ticksuffix = 'B/s',
    ),
    xaxis = dict(title=None)
)
fig.show()

In [101]:
fig = px.bar(df[df.Group=="DoubleFieldParse"], x='Variant', y='bytes_per_second', title="String to Double")
fig.update_layout(
    yaxis = dict(
        title=None,
        ticksuffix = 'B/s',
    ),
    xaxis = dict(title=None)
)
fig.show()

In [106]:
data = df[df["Group"].isin(["ScanSpeed", "SplitLines"])]

fig = px.bar(data, x='Benchmark', y='bytes_per_second', title="Split large string into lines")
fig.update_layout(
    yaxis = dict(
        title=None,
        ticksuffix = 'B/s',
    ),
    xaxis = dict(title=None)
)
fig.show()


In [103]:
data = df[df["Group"].isin(["BlockParse"])]

fig = px.bar(data, x='Variant', y='bytes_per_second', title="Parse lines composed of [int] [int] [double]")
fig.update_layout(
    yaxis = dict(
        title=None,
        ticksuffix = 'B/s',
    ),
    xaxis = dict(title=None)
)
fig.show()